# Descriptors for Reusable @property Methods

This example is adpated from [Effictive Python](http://www.effectivepython.com/) by Brett Slatkin

The problem with @property methods is reuse. The methods it decorates cannot be reused for multiple attributes of the same class. Consider the follwing class which tracks the exam grades for various subjects and validates the grade is between 0 and 100:

In [1]:
class Exam:
    
    def __init__(self):
        self._math_grade = 0
    
    @property
    def math_grade(self):
        return self._math_grade
    
    @math_grade.setter
    def math_grade(self, value):
        if not(0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        self._math_grade = value

In [2]:
harpal = Exam()
harpal.math_grade = 70

If we add another subject it will look like:

In [3]:
class Exam:
    
    def __init__(self):
        self._english_grade = 0
        self._math_grade = 0
    
    @staticmethod
    def _check_grade(value):
        if not(0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')

    @property
    def english_grade(self):
        return self._math_grade
    
    @english_grade.setter
    def english_grade(self, value):
        self._check_grade(value)
        self._math_grade = value
    
    @property
    def math_grade(self):
        return self._math_grade
    
    @math_grade.setter
    def math_grade(self, value):
        self._check_grade(value)
        self._math_grade = value

In [4]:
harpal = Exam()
harpal.math_grad = 70
harpal.english_grade = 80

As you can see this is getting tedious very quickly. Each new subject will need a `@property` method and validation. 

A better way to do this would be with a `descriptor`. A descriptor class can provide `__get__` and `__set__` methods that allow to reuse the grade validation. Below is a `Grade` class which implements the `__get__` and `__set__` methods. The `Exam` class then has instances of the `Grade` class per subject.


In [5]:
class Grade:
    
    def __init__(self):
        self._value = 0
    
    def __get__(self, instance,  instance_type):
        return self._value
    
    def __set__(self, instance, value):
        if not(0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        self._value = value

class Exam:
    
    math = Grade()
    english = Grade()

In [6]:
harpal = Exam()
harpal.math = 70
harpal.english = 80

In [7]:
harpal.math

70

In [8]:
harpal.english

80

When a grade is set it is interpreted as:

In [9]:
Exam.__dict__['math'].__set__(harpal, 70)

And when you try to retrieve a grade it will be interpreted as: 

In [10]:
Exam.__dict__['math'].__get__(harpal, Exam)

70

This works well but can lead to unusual behaviour. Accessing attributes on a single `Exam` instance works as expected:


In [11]:
harpal.math

70

In [12]:
harpal.english

80

The issue arises on multiple instances of `Exam`

In [13]:
alice = Exam()

In [14]:
alice.math

70

In [15]:
alice.english

80

Alice already has scores on a fresh new instance of the Exam class! In fact she has the same scores as Harpal. The problem is the `Grade` instance is shared across all `Exam` instances for the attributes `math` and `english`. The `Grade` instance for `math` and `english` is created once in the program lifetime when the `Exam` class is first defined, not each time an `Exam` instance is created.

To solve this, we need the Grade class to keep track of its value for each unique Exam instance. I can do this by saving the per-instance state in a dictionary, but this will leak memory by holding an instance of every Exam ever
passed to `__set__` over the lifetime of the program.

To fix this, I can use Python’s `weakref` built-in module. This module provides a special
class called `WeakKeyDictionary` that can take the place of the simple dictionary used
for _values. The unique behavior of `WeakKeyDictionary` is that it will remove
`Exam` instances from its set of keys when the runtime knows it’s holding the instance’s
last remaining reference in the program. Python will do the bookkeeping for you and
ensure that the _values dictionary will be empty when all Exam instances are no longer
in use.

In [16]:
from weakref import WeakKeyDictionary

class Grade:
    
    def __init__(self):
        self._values = WeakKeyDictionary()
    
    def __get__(self, instance,  instance_type):
        if instance is None: 
            return self
        return self._values.get(instance, 0)
    
    def __set__(self, instance, value):
        if not(0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        self._values[instance] = value

class Exam:
    
    math = Grade()
    english = Grade()

In [17]:
harpal = Exam()
harpal.math = 88
harpal.english = 75

In [18]:
harpal.math

88

In [19]:
harpal.english

75

In [20]:
alice = Exam()

In [21]:
alice.math

0

In [22]:
alice.english

0

In [23]:
alice.math = 90

In [24]:
alice.english = 95

Now, everything works as expected :) with multiple instances of the `Exam` class